In [1]:
import pandas as pd
import numpy as np
from DS_Planck_Unet import val_pix
from DS_models_stat import stat_orig_cats, stat_split_cats

In [2]:
files_full = {'pz14' : '/home/rt2122/Data/detected_cats/full_pz14_thr0.1_step8.csv', 
        'pz40' : '/home/rt2122/Data/detected_cats/full_pz40_thr0.1_step8.csv',
        'pz_act10' : '/home/rt2122/Data/detected_cats/full_pz_act10_thr0.1_step8.csv',
        'pz_act14' : '/home/rt2122/Data/detected_cats/full_pz_act20_thr0.1_step8.csv',
        'pz_act_rot_drop0.1_ep9' : 
              '/home/rt2122/Data/detected_cats/full_pz_act_rot_drop0.1_ep9_thr0.1_step8.csv',
        'pz_act_jan_rot_drop0.1_ep6' : 
              '/home/rt2122/Data/detected_cats/full_pz_act_jan_rot_drop0.1_ep6_thr0.1_step8.csv',
        'pz_act_feb_rot_drop0.1_ep5' : 
             '/home/rt2122/Data/detected_cats/full_pz_act_feb_rot_drop0.1_ep5_thr0.1_step8.csv'}

In [3]:
res_df = stat_split_cats(files_full, val_pix)
res_df

,name,planck_z,planck_no_z,mcxcwp,actwp,fp
0,pz14,0.939394,0.821429,0.155340,0.130435,949
1,pz40,0.931818,0.785714,0.145631,0.130435,1180
2,pz_act10,0.901515,0.821429,0.097087,0.089202,624
3,pz_act14,0.825758,0.750000,0.058252,0.043478,557
4,pz_act_rot_drop0.1_ep9,0.931818,0.750000,0.155340,0.094518,605
5,pz_act_jan_rot_drop0.1_ep6,0.954545,0.785714,0.106796,0.093750,693
6,pz_act_feb_rot_drop0.1_ep5,0.969697,0.821429,0.116505,0.163603,1843


In [4]:
res_df.to_csv('/home/rt2122/Data/models_stat/by_split_cats.csv', index=False)

In [5]:
comp_df, recall_df = stat_orig_cats(files_full, '/home/rt2122/Data/original_catalogs/csv', 
                                    big_pix=val_pix, dict_cut={})

/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


In [6]:
recall_df

,PSZ2,MCXC,RM,ACT,fp,all
pz14,0.93125,0.439759,0.043817,0.220930,949,1181
pz40,0.91875,0.421687,0.048361,0.219269,1180,1409
pz_act10,0.90000,0.391566,0.035054,0.159468,624,814
pz_act14,0.83125,0.355422,0.029211,0.121262,557,716
pz_act_rot_drop0.1_ep9,0.91875,0.427711,0.037326,0.186047,605,815
pz_act_jan_rot_drop0.1_ep6,0.93750,0.409639,0.038624,0.179402,693,903
pz_act_feb_rot_drop0.1_ep5,0.96250,0.433735,0.056151,0.242525,1843,2095


In [7]:
comp_df

,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,RM,RM_err,RM_std,ACT,ACT_err,ACT_std,all,fp
pz14,149,0.625,0.056014,73,0.605,0.053939,135,14.395,0.270296,133,2.095,0.106473,1181,949
pz40,147,0.830,0.059356,70,0.830,0.065781,149,17.260,0.273929,132,2.885,0.117593,1409,1180
pz_act10,144,0.585,0.052704,65,0.585,0.054116,108,9.255,0.223382,96,1.665,0.093582,814,624
pz_act14,133,0.395,0.046967,59,0.325,0.038787,90,8.140,0.192972,73,1.480,0.082657,716,557
pz_act_rot_drop0.1_ep9,147,0.555,0.049974,71,0.520,0.049092,115,9.910,0.220118,112,1.705,0.093722,815,605
pz_act_jan_rot_drop0.1_ep6,150,0.555,0.048442,68,0.470,0.050080,119,10.250,0.223466,108,1.780,0.096460,903,693
pz_act_feb_rot_drop0.1_ep5,154,1.365,0.082419,72,1.265,0.083117,173,25.835,0.381861,146,4.105,0.158732,2095,1843
all,160,0.000,NaN,166,0.000,NaN,3081,0.000,NaN,602,0.000,NaN,0,0


In [8]:
recall_df.to_csv('/home/rt2122/Data/models_stat/by_orig_cats.csv', index=False)

In [9]:
comp_df, recall_df = stat_orig_cats(files_full, '/home/rt2122/Data/original_catalogs/csv', 
                                    big_pix=list(range(48)), dict_cut={})

/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


In [10]:
comp_df

,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,RM,RM_err,RM_std,ACT,ACT_err,ACT_std,all,fp
pz14,1490,14.530,0.272888,730,15.965,0.274583,1266,282.760,1.114009,848,41.835,0.443307,16729,14594
pz40,1503,16.960,0.284953,740,19.865,0.295840,1315,322.920,1.187505,881,48.935,0.495714,19495,17306
pz_act10,1360,13.585,0.256011,662,13.685,0.268401,1058,206.545,1.021694,940,31.055,0.412688,18363,16316
pz_act14,1219,9.860,0.210770,602,11.035,0.236428,1230,190.720,0.911131,1884,31.875,0.393090,12239,9398
pz_act_rot_drop0.1_ep9,1437,11.795,0.235141,683,12.715,0.246725,1071,203.375,1.066376,779,29.870,0.406066,14224,12236
pz_act_jan_rot_drop0.1_ep6,1482,13.210,0.247864,707,14.515,0.263812,1122,224.275,1.037753,784,33.600,0.426096,17210,15162
pz_act_feb_rot_drop0.1_ep5,1513,25.260,0.345774,755,27.690,0.392927,1631,475.300,1.352199,958,71.575,0.604399,28781,26479
all,1653,0.000,NaN,1743,0.000,NaN,26111,0.000,NaN,4195,0.000,NaN,0,0


In [11]:
recall_df

,PSZ2,MCXC,RM,ACT,fp,all
pz14,0.901391,0.418818,0.048485,0.202145,14594,16729
pz40,0.909256,0.424555,0.050362,0.210012,17306,19495
pz_act10,0.822747,0.379805,0.040519,0.224076,16316,18363
pz_act14,0.737447,0.345382,0.047107,0.449106,9398,12239
pz_act_rot_drop0.1_ep9,0.869328,0.391853,0.041017,0.185697,12236,14224
pz_act_jan_rot_drop0.1_ep6,0.896552,0.405622,0.042970,0.186889,15162,17210
pz_act_feb_rot_drop0.1_ep5,0.915306,0.433161,0.062464,0.228367,26479,28781


In [12]:
recall_df.to_csv('/home/rt2122/Data/models_stat/by_orig_cats_full.csv', index=False)